# Corporate Action Service - Dividends & Events

Track corporate actions including dividend announcements, shareholder meetings (AGM/EGM), and other corporate events.

## Overview

The **Corporate Action Service** tracks important corporate events:

- **Dividend Tracking**: XD dates, payment dates, dividend amounts
- **Shareholder Meetings**: AGM (Annual General Meeting) and EGM (Extraordinary General Meeting)
- **Meeting Details**: Agenda, venue, meeting dates
- **Dividend Capture**: Plan dividend capture strategies
- **Calendar Management**: Track upcoming corporate events

### When to Use This Service

- Dividend income strategies
- Shareholder rights management
- Corporate event calendars
- Ex-dividend date tracking
- Meeting participation planning

In [1]:
!pip install settfex

zsh:1: command not found: pip


In [2]:
import asyncio
from settfex.services.set import get_corporate_actions
import pandas as pd
from datetime import datetime

print("Libraries imported!")

Libraries imported!


## Basic Usage

In [3]:
# Fetch corporate actions
actions = await get_corporate_actions("AOT")

print(f"Found {len(actions)} corporate actions for AOT\n")

for action in actions[:5]:  # Show first 5
    print(f"Type: {action.ca_type}")
    print(f"XD Date: {action.x_date}")
    
    if action.ca_type == "XD":  # Dividend
        print(f"Dividend: {action.dividend} {action.currency}")
        print(f"Payment Date: {action.payment_date}")
        print(f"Source: {action.source_of_dividend}")
    elif action.ca_type == "XM":  # Meeting
        print(f"Meeting: {action.meeting_type}")
        print(f"Date: {action.meeting_date}")
        print(f"Agenda: {action.agenda}")
    
    print("-" * 60)

2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'AOT' (lang=en) from https://www.set.or.th/api/set/stock/AOT/corporate-action?lang=en
2025-10-05 21:08:51 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 21:08:51 | INFO     | settfex.utils.session_manager:__init__:98 | SessionManager created with browser=chrome120, warmup_site=set, cache=enabled
2025-10-05 21:08:51 | INFO     | settfex.utils.session_cache:__init__:76 | SessionCache initialized at /Users/sarat/.settfex/cache (ttl=3600s, size_limit=100MB)
2025-10-05 21:08:51 | SUCCESS  | settfex.utils.session_manager:_try_load_from_cache:202 | ✓ Loaded session from cache: 4 cookies (age

Found 8 corporate actions for AOT

Type: XD
XD Date: 2024-12-04 00:00:00+07:00
Dividend: 0.79 Baht
Payment Date: 2025-02-06 00:00:00+07:00
Source: Net Profit
------------------------------------------------------------
Type: XM
XD Date: 2024-12-04 00:00:00+07:00
Meeting: AGM
Date: 2025-01-24 14:00:00+07:00
Agenda: Cash dividend payment,Changing The director(s)
------------------------------------------------------------
Type: XD
XD Date: 2023-12-08 00:00:00+07:00
Dividend: 0.36 Baht
Payment Date: 2024-02-14 00:00:00+07:00
Source: Net Profit
------------------------------------------------------------
Type: XM
XD Date: 2023-12-08 00:00:00+07:00
Meeting: AGM
Date: 2024-01-30 14:00:00+07:00
Agenda: Cash dividend payment,Changing the director(s)
------------------------------------------------------------
Type: XM
XD Date: 2022-12-09 00:00:00+07:00
Meeting: AGM
Date: 2023-01-20 14:00:00+07:00
Agenda: Omitted dividend payment,Changing the director(s)
----------------------------------------

## Advanced Usage - Dividend Calendar

In [4]:
async def build_dividend_calendar(symbols: list[str]):
    """
    Build a dividend calendar for multiple stocks.
    """
    tasks = [get_corporate_actions(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    dividends = []
    for symbol, actions in zip(symbols, results):
        if isinstance(actions, Exception):
            continue
        
        for action in actions:
            if action.ca_type == "XD" and action.dividend:
                dividends.append({
                    "symbol": symbol,
                    "xd_date": action.x_date,
                    "payment_date": action.payment_date,
                    "dividend": action.dividend,
                    "currency": action.currency,
                    "source": action.source_of_dividend,
                    "type": action.dividend_type
                })
    
    df = pd.DataFrame(dividends)
    if not df.empty:
        df = df.sort_values('xd_date', ascending=False)
    return df

# Build calendar
symbols = ["PTT", "KBANK", "CPALL", "AOT", "BBL"]
div_calendar = await build_dividend_calendar(symbols)

print(f"Dividend Calendar ({len(div_calendar)} events):\n")
print(div_calendar.head(20).to_string(index=False))

2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'PTT' (lang=en) from https://www.set.or.th/api/set/stock/PTT/corporate-action?lang=en
2025-10-05 21:08:51 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:08:51 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'KBANK' (lang=en) from https://www.set.or.th/api/set/stock/KBANK/corporate-action?lang=en
2025-10-05 21:08:51 | INFO     | settfex.utils.data

Dividend Calendar (44 events):

symbol                   xd_date              payment_date  dividend currency                           source          type
   PTT 2025-10-01 00:00:00+07:00 2025-10-17 00:00:00+07:00      0.90     Baht Net profit and retained earnings Cash Dividend
 KBANK 2025-09-10 00:00:00+07:00 2025-09-26 00:00:00+07:00      2.00     Baht                Retained Earnings Cash Dividend
   BBL 2025-09-10 00:00:00+07:00 2025-09-26 00:00:00+07:00      2.00     Baht                Retained Earnings Cash Dividend
 KBANK 2025-05-15 00:00:00+07:00 2025-06-06 00:00:00+07:00      2.50     Baht                       Net Profit Cash Dividend
 CPALL 2025-05-06 00:00:00+07:00 2025-05-23 00:00:00+07:00      1.35     Baht                       Net Profit Cash Dividend
   BBL 2025-04-23 00:00:00+07:00 2025-05-09 00:00:00+07:00      6.50     Baht                Retained Earnings Cash Dividend
 KBANK 2025-04-17 00:00:00+07:00 2025-05-09 00:00:00+07:00      8.00     Baht                

## Use Case: Upcoming Dividends

In [ ]:
def find_upcoming_dividends(div_calendar, days_ahead: int = 30):
    """
    Find dividends with XD dates in the next N days.
    """
    today = pd.Timestamp(datetime.now().date())  # Timezone-naive
    
    # Filter upcoming
    df = div_calendar.copy()
    df['xd_date'] = pd.to_datetime(df['xd_date']).dt.tz_localize(None)  # Remove timezone
    df['days_until'] = (df['xd_date'] - today).dt.days
    
    upcoming = df[df['days_until'].between(0, days_ahead)].sort_values('days_until')
    return upcoming

# Find upcoming dividends
upcoming = find_upcoming_dividends(div_calendar, days_ahead=60)
print(f"Upcoming Dividends (next 60 days):\n")
if not upcoming.empty:
    print(upcoming[['symbol', 'xd_date', 'dividend', 'payment_date', 'days_until']].to_string(index=False))
else:
    print("No upcoming dividends found")

## Use Case: Meeting Tracker

In [6]:
async def track_shareholder_meetings(symbols: list[str]):
    """
    Track upcoming shareholder meetings (AGM/EGM).
    """
    tasks = [get_corporate_actions(symbol) for symbol in symbols]
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    meetings = []
    for symbol, actions in zip(symbols, results):
        if isinstance(actions, Exception):
            continue
        
        for action in actions:
            if action.ca_type == "XM":  # Meeting
                meetings.append({
                    "symbol": symbol,
                    "meeting_type": action.meeting_type,
                    "meeting_date": action.meeting_date,
                    "book_close": action.book_close_date,
                    "agenda": action.agenda[:50] if action.agenda else "N/A"  # Truncate
                })
    
    return pd.DataFrame(meetings).sort_values('meeting_date', ascending=False)

meetings = await track_shareholder_meetings(symbols)
print(f"Shareholder Meetings ({len(meetings)}):\n")
print(meetings.head(10).to_string(index=False))

2025-10-05 21:13:05 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:13:05 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'PTT' (lang=en) from https://www.set.or.th/api/set/stock/PTT/corporate-action?lang=en
2025-10-05 21:13:05 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 21:13:05 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:13:05 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'KBANK' (lang=en) from https://www.set.or.th/api/set/stock/KBANK/corporate-action?lang=en
2025-10-05 21:13:05 | INFO     | settfex.utils.data

Shareholder Meetings (33):

symbol meeting_type              meeting_date book_close                                             agenda
 KBANK          EGM 2025-05-07 14:00:00+07:00       None Cash dividend payment,To consider approving the am
 CPALL          AGM 2025-04-25 14:00:00+07:00       None     Cash dividend payment,Changing The director(s)
   BBL          AGM 2025-04-11 15:00:00+07:00       None Cash dividend payment,The issuance of debentures,C
   PTT          AGM 2025-04-11 13:30:00+07:00       None     Cash dividend payment,Changing The director(s)
 KBANK          AGM 2025-04-09 14:00:00+07:00       None Cash dividend payment,Changing The director(s),To 
   AOT          AGM 2025-01-24 14:00:00+07:00       None     Cash dividend payment,Changing The director(s)
 CPALL          AGM 2024-04-26 14:00:00+07:00       None                              Cash dividend payment
   PTT          AGM 2024-04-12 13:30:00+07:00       None     Cash dividend payment,Changing The director(s)


## Use Case: Dividend Yield Analysis

In [7]:
# Calculate annual dividend totals
annual_divs = div_calendar.groupby('symbol')['dividend'].sum().reset_index()
annual_divs.columns = ['symbol', 'total_dividend']
annual_divs = annual_divs.sort_values('total_dividend', ascending=False)

print("Total Dividends by Stock (historical):")
print(annual_divs.to_string(index=False))

Total Dividends by Stock (historical):
symbol  total_dividend
 KBANK           34.75
   BBL           33.00
   PTT           11.10
 CPALL            5.85
   AOT            1.34


## Error Handling

In [8]:
async def safe_get_corporate_actions(symbol: str):
    try:
        actions = await get_corporate_actions(symbol)
        print(f"✓ {symbol}: {len(actions)} actions")
        return actions
    except Exception as e:
        print(f"✗ {symbol}: {e}")
        return []

await safe_get_corporate_actions("PTT")

2025-10-05 21:13:12 | INFO     | settfex.services.set.stock.corporate_action:__init__:108 | CorporateActionService initialized with base_url=https://www.set.or.th
2025-10-05 21:13:12 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:151 | Fetching corporate actions for symbol 'PTT' (lang=en) from https://www.set.or.th/api/set/stock/PTT/corporate-action?lang=en
2025-10-05 21:13:12 | INFO     | settfex.utils.data_fetcher:__init__:113 | AsyncDataFetcher initialized with browser=chrome120, timeout=30s, rate_limit=0.0s
2025-10-05 21:13:12 | INFO     | settfex.utils.data_fetcher:fetch:303 | Fetch successful: url=https://www.set.or.th/api/set/stock/PTT/corporate-action?lang=en, status=200, elapsed=0.01s, size=8913 bytes
2025-10-05 21:13:12 | INFO     | settfex.services.set.stock.corporate_action:fetch_corporate_actions:190 | Successfully fetched 19 corporate action(s) for PTT


✓ PTT: 19 actions


[CorporateAction(symbol='PTT', name='', ca_type='XD', type='XD', book_close_date=None, record_date=datetime.datetime(2025, 10, 2, 0, 0, tzinfo=TzInfo(+07:00)), remark=None, x_date=datetime.datetime(2025, 10, 1, 0, 0, tzinfo=TzInfo(+07:00)), x_session='', payment_date=datetime.datetime(2025, 10, 17, 0, 0, tzinfo=TzInfo(+07:00)), begin_operation=datetime.datetime(2025, 1, 1, 0, 0, tzinfo=TzInfo(+07:00)), end_operation=datetime.datetime(2025, 6, 30, 0, 0, tzinfo=TzInfo(+07:00)), source_of_dividend='Net profit and retained earnings', dividend=0.9, currency='Baht', ratio=None, dividend_type='Cash Dividend', approximate_payment_date=None, tentative_dividend_flag=None, tentative_dividend=None, dividend_payment='0.90', meeting_date=None, agenda=None, venue=None, meeting_type=None, inquiry_date=None),
 CorporateAction(symbol='PTT', name='', ca_type='XD', type='XD', book_close_date=None, record_date=datetime.datetime(2025, 3, 7, 0, 0, tzinfo=TzInfo(+07:00)), remark=None, x_date=datetime.datetime

## Next Steps

- **[Highlight Data](02_highlight_data.ipynb)** - Current dividend yield
- **[Shareholder](06_shareholder.ipynb)** - Ownership structure
- **[Financial](11_financial.ipynb)** - Financial statements

**Documentation**: `/docs/settfex/services/set/corporate_action.md`